# sentiment dictionary

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
def tf_extractor(corpus):  
    # returns a frequency-based DTM
    vectorizer = CountVectorizer(min_df=1, ngram_range=(1,1)) 
    features = vectorizer.fit_transform(corpus) # transform texts to a frequency matrix
    return vectorizer, features  

In [3]:
def tfidf_extractor(corpus):
    # returns a tf-idf based DTM
    vectorizer = TfidfVectorizer(min_df=1, 
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True,
                                 ngram_range=(1,1))
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

## <br>
## naver_clova

In [112]:
path = r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data'
data = pd.read_csv(path + r'\playstore_review_naver_clova.csv', encoding='UTF-8')
comments = list(data['comment'])
ratings = list(data['ratings'])
data.shape

(2556, 5)

In [113]:
filtered_texts = []
filtered_labels = []

for text, score in zip(comments, ratings):
    if score == 3:
        continue
        
    # 평점 기준으로 문서에 label을 부여
    # 1 ~ 2 -> 부정, -1
    # 4 ~ 5 -> 긍정, 1
    filtered_texts.append(text)
    filtered_labels.append(1 if score > 3 else -1)

In [114]:
# You can also use the following method
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.3, random_state=0)

In [115]:
tf_vectorizer, train_tf_features = tf_extractor(train_texts)
# input의 형태 = list of docs
test_tf_features = tf_vectorizer.transform(test_texts)
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]
# tf_vectorizer.vocabulary_.items() returns a list of (word, frequency)
# We sort words based on their frequencies and save the words

In [119]:
# tf matrix를 사용한 경우
lr = LogisticRegression(C=0.1, penalty='l2', solver='sag') # Lasso regression
# C = Inverse of regularization strength, 즉 C 값이 작을수록 penalty를 많이 준다는 것입니다.
# penalty를 많이 준다는 뜻은 L1 같은 경우는 feature의 수를 그만큼 많이 줄인다는 뜻이고
# L2인 경우는 weight 값을 더 0에 가깝게 한다는 뜻입니다.
lr.fit(train_tf_features, train_labels) # 학습
pred_labels = lr.predict(test_tf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 178 out of 641
Accuracy: 0.72


In [38]:
# tfidf matrix를 사용한 경우
tfidf_vectorizer, train_tfidf_features = tfidf_extractor(train_texts)
test_tfidf_features = tfidf_vectorizer.transform(test_texts)
lr = LogisticRegression(C=0.1, penalty='l1', solver='saga') # Lasso regression
lr.fit(train_tfidf_features, train_labels) # 학습
pred_labels = lr.predict(test_tfidf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 304 out of 641
Accuracy: 0.53


In [41]:
# Get coefficients of the model
coefficients = lr.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(7574, 1.039721417408199), (7600, 0.7724308659319985), (7561, 0.5665838283947136), (1049, 0.47337489103995595), (7547, 0.4468903456873619)]
좋아요 (1.040)
좋은데 (0.772)
좋네요 (0.567)
근데 (0.473)
좋겠어요 (0.447)
좋겠네요 (0.432)
좋겠습니다 (0.384)
같아요 (0.351)
있게 (0.320)
좋습니다 (0.318)
부르면 (0.315)
감사합니다 (0.311)
ㅎㅎ (0.302)
빅스비 (0.284)
있었으면 (0.278)
추가해주세요 (0.273)
하지만 (0.267)
합니다 (0.260)
해주세요 (0.252)
되면 (0.249)
있으면 (0.238)
good (0.231)
정말 (0.230)
빅스비보다 (0.219)
다좋은데 (0.218)
너무좋아요 (0.210)
클로바를 (0.209)
저는 (0.208)
시리처럼 (0.205)
좋고 (0.204)
좋은 (0.203)
재생이 (0.203)
더욱 (0.198)
좋음 (0.196)
주세요 (0.185)
좋을것 (0.184)
한가지 (0.183)
최고 (0.178)
있는데요 (0.173)
굿굿 (0.167)
연동 (0.159)
같습니다 (0.157)
하네요 (0.156)
아주 (0.155)
에서 (0.152)
쓰고 (0.151)
음성호출이 (0.148)
인공지능 (0.147)
이름이 (0.146)
좋겠다 (0.145)
샀는데 (-0.190)
도대체 (-0.194)
앱을 (-0.197)
음성 (-0.197)
같은데 (-0.198)
어플도 (-0.199)
스피커도 (-0.205)
혼자 (-0.207)
갑자기 (-0.208)
나오네요 (-0.209)
점점 (-0.209)
그냥 (-0.210)
없음 (-0.219)
해도 (-0.221)
안되는게 (-0.221)
어떻게 (-0.225)
너무 (-0.229)
프렌즈 (-0.229)
클로바 (-0.233)
않는 (-0.2

In [64]:
with open(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\sentiment_dictionary\naver_clova.txt', 'w', encoding='utf-8') as f:
    for word, coef in sorted_coefficients:
        f.write(vocablist[word]+'\t'+str(coef)+'\n')
f.close()

## <br>
## kakao_mini

In [65]:
path = r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data'
data = pd.read_csv(path + r'\playstore_review_kakao_mini.csv', encoding='UTF-8')
comments = list(data['comment'])
ratings = list(data['ratings'])
data.shape

(578, 5)

In [66]:
filtered_texts = []
filtered_labels = []

for text, score in zip(comments, ratings):
    if score == 3:
        continue
        
    filtered_texts.append(text)
    filtered_labels.append(1 if score > 3 else -1)

In [67]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.3, random_state=0)
tf_vectorizer, train_tf_features = tf_extractor(train_texts)
test_tf_features = tf_vectorizer.transform(test_texts)
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]

In [71]:
# tf matrix를 사용한 경우
lr = LogisticRegression(C=0.1, penalty='l2', solver='sag') # Lasso regression
# C = Inverse of regularization strength, 즉 C 값이 작을수록 penalty를 많이 준다는 것입니다.
# penalty를 많이 준다는 뜻은 L1 같은 경우는 feature의 수를 그만큼 많이 줄인다는 뜻이고
# L2인 경우는 weight 값을 더 0에 가깝게 한다는 뜻입니다.
lr.fit(train_tf_features, train_labels) # 학습
pred_labels = lr.predict(test_tf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 50 out of 149
Accuracy: 0.66


In [69]:
# tfidf matrix를 사용한 경우
tfidf_vectorizer, train_tfidf_features = tfidf_extractor(train_texts)
test_tfidf_features = tfidf_vectorizer.transform(test_texts)
lr = LogisticRegression(C=0.1, penalty='l1', solver='saga') # Lasso regression
lr.fit(train_tfidf_features, train_labels) # 학습
pred_labels = lr.predict(test_tfidf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 69 out of 149
Accuracy: 0.54


In [72]:
# Get coefficients of the model
coefficients = lr.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(2523, 0.607919781186865), (2512, 0.3326869482487577), (2511, 0.25866577529010176), (2708, 0.24190978331699442), (2516, 0.18162850739750763)]
좋아요 (0.608)
좋겠어요 (0.333)
좋겠습니다 (0.259)
카카오미니 (0.242)
좋네요 (0.182)
기능이 (0.180)
좋고 (0.164)
음악 (0.159)
좋은데 (0.158)
있으면 (0.146)
업데이트 (0.145)
제가 (0.142)
ㅠㅠ (0.117)
기대합니다 (0.114)
근데 (0.112)
있습니다 (0.110)
노래도 (0.107)
잘쓰고 (0.106)
너무 (0.105)
유용하게 (0.105)
위젯이 (0.104)
부탁드려요 (0.104)
좋다 (0.102)
헤이카카오 (0.102)
지금은 (0.101)
해주세요 (0.098)
있어서 (0.097)
원하는 (0.096)
좋습니다 (0.095)
아쉽네요 (0.091)
제발 (0.090)
영어로 (0.089)
그게 (0.087)
베이스 (0.087)
음질도 (0.083)
신기하고 (0.080)
다른건 (0.080)
좋은데요 (0.080)
같아요 (0.079)
기능 (0.078)
앞으로 (0.078)
반응도 (0.076)
만족합니다 (0.075)
있었으면 (0.075)
하지 (0.074)
설정이 (0.074)
재생할때 (0.072)
빠른 (0.072)
다만 (0.072)
최고 (0.072)
미니 (-0.088)
사용 (-0.088)
같은 (-0.088)
와이파이도 (-0.089)
안잡히고 (-0.091)
이게 (-0.094)
잡음이 (-0.095)
불러도 (-0.095)
와이파이를 (-0.097)
없다고 (-0.099)
해결해주세요 (-0.102)
연결도 (-0.103)
공유기 (-0.104)
안되는데 (-0.104)
카카오 (-0.105)
있어요 (-0.105)
무슨 (-0.109)
불편해요 (-0.112)
바로 (-0.11

In [73]:
with open(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\sentiment_dictionary\kakao_mini.txt', 'w', encoding='utf-8') as f:
    for word, coef in sorted_coefficients:
        f.write(vocablist[word]+'\t'+str(coef)+'\n')
f.close()

## <br>
## kt_gigagenie

In [74]:
path = r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data'
data = pd.read_csv(path + r'\playstore_review_kt_gigagenie.csv', encoding='UTF-8')
comments = list(data['comment'])
ratings = list(data['ratings'])
data.shape

(430, 5)

In [75]:
filtered_texts = []
filtered_labels = []

for text, score in zip(comments, ratings):
    if score == 3:
        continue
        
    filtered_texts.append(text)
    filtered_labels.append(1 if score > 3 else -1)

In [76]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.3, random_state=0)
tf_vectorizer, train_tf_features = tf_extractor(train_texts)
test_tf_features = tf_vectorizer.transform(test_texts)
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]

In [79]:
# tf matrix를 사용한 경우
lr = LogisticRegression(C=0.1, penalty='l2', solver='sag') # Lasso regression
lr.fit(train_tf_features, train_labels) # 학습
pred_labels = lr.predict(test_tf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 32 out of 110
Accuracy: 0.71


In [78]:
# tfidf matrix를 사용한 경우
tfidf_vectorizer, train_tfidf_features = tfidf_extractor(train_texts)
test_tfidf_features = tfidf_vectorizer.transform(test_texts)
lr = LogisticRegression(C=0.1, penalty='l1', solver='saga') # Lasso regression
lr.fit(train_tfidf_features, train_labels) # 학습
pred_labels = lr.predict(test_tfidf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 55 out of 110
Accuracy: 0.50


In [80]:
# Get coefficients of the model
coefficients = lr.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(1142, 0.6547985427460136), (1224, 0.28839288216950926), (1151, 0.21039604604230958), (1282, 0.13418928234346778), (460, 0.11387123016243433)]
좋아요 (0.655)
최고 (0.288)
좋음 (0.210)
편해요 (0.134)
매일 (0.114)
kt간편로그인에 (0.112)
좋다 (0.092)
좋네요 (0.091)
good (0.087)
좋은 (0.084)
듭니다 (0.083)
아쉽네요 (0.082)
간편하고 (0.079)
제가 (0.071)
아주 (0.064)
우리집 (0.063)
근데 (0.059)
등록된 (0.056)
아이디가 (0.056)
아이디를 (0.056)
좋습니다 (0.048)
취향 (0.048)
너무감사쥬쥬 (0.048)
말귀를잘알아들음 (0.048)
기가지니가티비아니면핸드폰에나오는데난기가지니를처음써본다 (0.048)
김정순 (0.048)
좋아좋아 (0.048)
넘좋아요ㅎㅎ (0.048)
교통상황빨리 (0.048)
편함 (0.048)
멋져뿌러 (0.048)
조찬복 (0.048)
재재가재재를먹어부렸어 (0.048)
좋아ㅡ (0.048)
네좋아요 (0.048)
다좋음 (0.048)
넘냐ㅡㅎ은것 (0.048)
좋을것갔다 (0.048)
좋아여 (0.048)
지니를궃이않불러도돼서편리해요 (0.048)
조아 (0.048)
헤헤 (0.048)
오케 (0.048)
감사 (0.048)
간단하네ㅎㅎ (0.048)
디ㅡㄱㅅㄱㅅ (0.048)
편리해요 (0.048)
ㅠㅠ5555 (0.048)
기가지니는참말을잘듣습니다 (0.048)
않되는데 (0.048)
쓰고 (-0.077)
굉장히 (-0.077)
지금 (-0.078)
담거나 (-0.079)
해주세요 (-0.079)
노래가 (-0.081)
정말 (-0.081)
권한 (-0.082)
장난까냐 (-0.082)
기능이 (-0.082)
늘려주세요 (-0.083)
ㅡㅡ (-0.083)
갑자기 (-0.083)
쓸데

In [81]:
with open(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\sentiment_dictionary\kt_gigagenie.txt', 'w', encoding='utf-8') as f:
    for word, coef in sorted_coefficients:
        f.write(vocablist[word]+'\t'+str(coef)+'\n')
f.close()

## <br>
## skt_nugu

In [82]:
path = r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data'
data = pd.read_csv(path + r'\playstore_review_skt_nugu.csv', encoding='UTF-8')
comments = list(data['comment'])
ratings = list(data['ratings'])
data.shape

(1940, 5)

In [83]:
filtered_texts = []
filtered_labels = []

for text, score in zip(comments, ratings):
    if score == 3:
        continue
        
    filtered_texts.append(text)
    filtered_labels.append(1 if score > 3 else -1)

In [84]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.3, random_state=0)
tf_vectorizer, train_tf_features = tf_extractor(train_texts)
test_tf_features = tf_vectorizer.transform(test_texts)
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]

In [87]:
# tf matrix를 사용한 경우
lr = LogisticRegression(C=0.1, penalty='l2', solver='sag') # Lasso regression
lr.fit(train_tf_features, train_labels) # 학습
pred_labels = lr.predict(test_tf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 134 out of 488
Accuracy: 0.73


In [86]:
# tfidf matrix를 사용한 경우
tfidf_vectorizer, train_tfidf_features = tfidf_extractor(train_texts)
test_tfidf_features = tfidf_vectorizer.transform(test_texts)
lr = LogisticRegression(C=0.1, penalty='l1', solver='saga') # Lasso regression
lr.fit(train_tfidf_features, train_labels) # 학습
pred_labels = lr.predict(test_tfidf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 175 out of 488
Accuracy: 0.64


In [88]:
# Get coefficients of the model
coefficients = lr.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(5314, 1.1386396414224107), (5308, 0.4173584755355908), (4866, 0.33210715718373546), (5311, 0.32803802155508194), (3434, 0.31923048287435984)]
좋아요 (1.139)
좋네요 (0.417)
있습니다 (0.332)
좋습니다 (0.328)
아리아 (0.319)
좋겠습니다 (0.307)
good (0.286)
좋겠어요 (0.276)
최고 (0.268)
기능이 (0.260)
좋음 (0.241)
감사합니다 (0.233)
있으면 (0.220)
좋은 (0.202)
좋은데 (0.199)
편해요 (0.195)
많이 (0.193)
사용하고 (0.175)
기대됩니다 (0.162)
기능은 (0.161)
생각보다 (0.148)
nugu (0.144)
너무좋아요 (0.144)
정말 (0.142)
랜덤 (0.141)
아주 (0.135)
있어요 (0.135)
기능들이 (0.133)
혹시 (0.129)
추가해주세요 (0.128)
있어서 (0.125)
제가 (0.122)
편리해요 (0.120)
있도록 (0.119)
아주좋아요 (0.118)
좋아 (0.117)
다좋은데 (0.116)
있었으면 (0.115)
난리 (0.115)
잘쓰고 (0.114)
최고의 (0.113)
있는데 (0.112)
크게 (0.112)
조아요 (0.111)
편하고 (0.111)
좋겠네요 (0.111)
있는 (0.109)
좀더 (0.107)
보입니다 (0.107)
유튜브 (0.106)
연결을 (-0.166)
끊겨서 (-0.167)
처럼 (-0.168)
사용 (-0.171)
안되요 (-0.171)
제대로 (-0.171)
접속이 (-0.171)
앱은 (-0.174)
멜론을 (-0.177)
진짜 (-0.178)
무조건 (-0.185)
아무리 (-0.185)
연결할수 (-0.186)
인터넷 (-0.188)
멜론 (-0.188)
없네요 (-0.188)
업데이트 (-0.188)
누르면 (-0.190)
말을 (-0.193)
나

In [89]:
with open(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\sentiment_dictionary\skt_nugu.txt', 'w', encoding='utf-8') as f:
    for word, coef in sorted_coefficients:
        f.write(vocablist[word]+'\t'+str(coef)+'\n')
f.close()

## <br>
## clova + nugu + gigagenie + mini

In [25]:
path = r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data'
data1 = pd.read_csv(path + r'\playstore_review_naver_clova.csv', encoding='UTF-8')
data2 = pd.read_csv(path + r'\playstore_review_skt_nugu.csv', encoding='UTF-8')
data3 = pd.read_csv(path + r'\playstore_review_kt_gigagenie.csv', encoding='UTF-8')
data4 = pd.read_csv(path + r'\playstore_review_kakao_mini.csv', encoding='UTF-8')
comments = list(data1['comment']) + list(data2['comment']) + list(data3['comment']) + list(data4['comment'])
ratings = list(data1['ratings']) + list(data2['ratings']) + list(data3['ratings']) + list(data4['ratings']) 

In [31]:
len(comments)

5504

In [26]:
filtered_texts = []
filtered_labels = []

for text, score in zip(comments, ratings):
    if score == 3:
        continue
        
    filtered_texts.append(text)
    filtered_labels.append(1 if score > 3 else -1)

In [64]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.3, random_state=0)
tf_vectorizer, train_tf_features = tf_extractor(train_texts)
test_tf_features = tf_vectorizer.transform(test_texts)
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]

In [28]:
tf_vectorizer.vocabulary_

{'저는': 12885,
 '멜론을': 5361,
 '사용해서': 7383,
 '1분미리듣기밖에': 85,
 '못하네요': 5675,
 '멜론도': 5329,
 '추가해주시면': 14367,
 '좋을것같아요': 13521,
 '한심한': 15689,
 '수준으로': 8079,
 '오류투성이': 10850,
 '특히': 14933,
 '음악재생시': 11532,
 '지멋대로': 13806,
 '얽혀서': 10048,
 '중단되고': 13665,
 '난리도': 2607,
 '아님': 8827,
 '클로바': 14686,
 '기기를': 1996,
 '샀고': 7476,
 '해보니': 15827,
 '다좋은데': 3491,
 '이거': 11586,
 '하나만': 15446,
 '추가해주세요': 14363,
 '폰에': 15257,
 '다운로드한': 3453,
 '음악': 11469,
 '플레이할수있는기능': 15354,
 '클로바한테': 14729,
 '틀어달라하니': 14960,
 '1분음악밖에': 87,
 '안나오더라구요': 9038,
 '들을려면': 4318,
 '돈내야하구': 3854,
 '구글': 1602,
 '플레이': 15342,
 '뮤직도': 5960,
 'ㅠㅜ': 707,
 '네이버뮤직': 2886,
 '연동': 10574,
 '어떻게하나요': 9836,
 '하는거': 15462,
 '보고': 6555,
 '지렸습니다': 13799,
 'ㄷㄷ': 662,
 '여기는': 10316,
 '창원': 14074,
 '마침': 4821,
 '투썸': 14899,
 '갈일이': 906,
 '있어서': 12330,
 '가까운': 742,
 '투썸플레이스': 14900,
 '가는': 754,
 '안내해줘': 9059,
 '자동차': 12412,
 '추천경로를': 14376,
 '알려드릴게요': 9497,
 '중부내륙고속도로': 13670,
 '당진영덕고속도로': 3624,
 '경부고속도로를': 1343,
 '거치는': 1181,
 '경로로': 1341,
 '359'

### Test accuracy <br>
- naver_clova : 0.72 (641) <br>
- skt_nugu : 0.73 (488) <br>
- kt_gigagenie : 0.71 (110) <br>
- kakao_mini : 0.66 (149) <br>

In [123]:
# C : 0.1 -> 1 변경시 accuracy 증가

In [65]:
# tf matrix를 사용한 경우
lr = LogisticRegression(C=1, penalty='l2', solver='sag') # Lasso regression
lr.fit(train_tf_features, train_labels) # 학습
pred_labels = lr.predict(test_tf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 298 out of 1387
Accuracy: 0.79


In [173]:
# tfidf matrix를 사용한 경우
tfidf_vectorizer, train_tfidf_features = tfidf_extractor(train_texts)
test_tfidf_features = tfidf_vectorizer.transform(test_texts)
lr = LogisticRegression(C=1, penalty='l1', solver='saga') # Lasso regression
lr.fit(train_tfidf_features, train_labels) # 학습
pred_labels = lr.predict(test_tfidf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 348 out of 1387
Accuracy: 0.75


In [33]:
# Get coefficients of the model
coefficients = lr.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(13457, 2.952741682091697), (13502, 2.2525192220030914), (372, 1.8490412614714824), (13439, 1.8172598497102472), (13525, 1.7247898491810476)]
좋아요 (2.953)
좋은데 (2.253)
good (1.849)
좋네요 (1.817)
좋음 (1.725)
좋습니다 (1.672)
최고 (1.620)
감사합니다 (1.575)
너무좋아요 (1.444)
근데 (1.327)
있습니다 (1.304)
아주좋아요 (1.235)
좋아 (1.183)
추가해주세요 (1.179)
다좋은데 (1.155)
편해요 (1.078)
좋고 (1.073)
감사 (1.064)
같아요 (1.045)
좋은 (1.038)
굿굿 (1.002)
제가 (0.988)
최고의 (0.987)
좋다 (0.979)
좋네 (0.957)
잘쓰고 (0.931)
최고에요 (0.913)
쵝오 (0.906)
좋겠네요 (0.905)
아리아 (0.902)
그런데 (0.891)
nugu (0.889)
조아요 (0.887)
빅스비보다 (0.883)
좋겠어요 (0.873)
있었으면 (0.866)
그래도 (0.845)
부탁드려요 (0.804)
해제 (0.804)
다만 (0.774)
없어도 (0.773)
되면 (0.763)
있는데요 (0.750)
유용하게 (0.749)
오류좀 (0.731)
신기하네요 (0.719)
좋겠습니다 (0.713)
가끔 (0.711)
기대합니다 (0.707)
잘되는데 (0.705)
말귀를 (-0.762)
답답하네요 (-0.764)
어플도 (-0.766)
티비 (-0.780)
모르겠어요 (-0.781)
구매했는데 (-0.791)
연동이 (-0.793)
절대 (-0.796)
한참 (-0.798)
네트워크가 (-0.809)
안되는데 (-0.812)
음성 (-0.820)
오류로 (-0.829)
말을 (-0.832)
쓰레기 (-0.843)
연결안됨 (-0.850)
스피커 (-0.859)
멈춤 (-0.866)
제발 (

In [34]:
for word, coef in sorted_coefficients:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

좋아요 (2.953)
좋은데 (2.253)
good (1.849)
좋네요 (1.817)
좋음 (1.725)
좋습니다 (1.672)
최고 (1.620)
감사합니다 (1.575)
너무좋아요 (1.444)
근데 (1.327)
있습니다 (1.304)
아주좋아요 (1.235)
좋아 (1.183)
추가해주세요 (1.179)
다좋은데 (1.155)
편해요 (1.078)
좋고 (1.073)
감사 (1.064)
같아요 (1.045)
좋은 (1.038)
굿굿 (1.002)
제가 (0.988)
최고의 (0.987)
좋다 (0.979)
좋네 (0.957)
잘쓰고 (0.931)
최고에요 (0.913)
쵝오 (0.906)
좋겠네요 (0.905)
아리아 (0.902)
그런데 (0.891)
nugu (0.889)
조아요 (0.887)
빅스비보다 (0.883)
좋겠어요 (0.873)
있었으면 (0.866)
그래도 (0.845)
부탁드려요 (0.804)
해제 (0.804)
다만 (0.774)
없어도 (0.773)
되면 (0.763)
있는데요 (0.750)
유용하게 (0.749)
오류좀 (0.731)
신기하네요 (0.719)
좋겠습니다 (0.713)
가끔 (0.711)
기대합니다 (0.707)
잘되는데 (0.705)
애들이 (0.704)
정말좋은데 (0.700)
좋은데요 (0.692)
있으면 (0.687)
클로바앱 (0.685)
좋은것 (0.680)
튕깁니다 (0.677)
한가지 (0.674)
와우 (0.673)
짱짱 (0.670)
유용한 (0.669)
하지만 (0.664)
고마워요 (0.661)
좋아좋아 (0.661)
굿잡 (0.661)
아주 (0.659)
잘쓰고있습니다 (0.659)
넘나 (0.657)
좋은것같아요 (0.645)
쓸만해요 (0.621)
필요해요 (0.619)
대기상태인데 (0.612)
안되요ㅜㅠ (0.612)
옆에있고 (0.612)
레알 (0.611)
속터져 (0.609)
좋아서 (0.608)
있었는데 (0.601)
그대로 (0.597)
합니다 (0.593)
없어서 (0.5

김결에게 (0.181)
분명 (0.181)
기계다룸이 (0.181)
어렵네요 (0.181)
조금은벅찬세대에접어들면서 (0.181)
추구하는 (0.181)
편리함만을 (0.181)
방법좀알려주세요 (0.181)
셋톱박스수동으로껴야되고요설명서도구체적이지않고 (0.181)
유플tv끄는것은되는데 (0.181)
켜는게안되는이유가뭘까요 (0.181)
클로바프렌즈미니 (0.181)
좋을것같아요 (0.181)
내꿈을 (0.180)
누웠는데 (0.180)
시작이야 (0.180)
심판자 (0.180)
여행 (0.180)
위한 (0.180)
피곤해서 (0.180)
피카츄 (0.180)
쓸수록 (0.180)
된다면 (0.180)
기독교방송이 (0.180)
라디오방송에 (0.180)
추가탑재 (0.180)
자꾸만 (0.180)
말로물어봐도 (0.180)
못알아먹었어요 (0.180)
이거어떻게해요 (0.180)
죄송합니다 (0.180)
키보드를치거나 (0.180)
nugu에서로그인해도 (0.179)
멜론계정을 (0.179)
3번이나요 (0.179)
다른노래들려줘 (0.179)
들려줘 (0.179)
받으려고 (0.179)
빠른노래 (0.179)
설문조사 (0.179)
알아먹어요 (0.179)
이벤트를 (0.179)
전곡 (0.179)
쿠폰 (0.179)
있을듯 (0.179)
나온다는데 (0.178)
날짜눌러도 (0.178)
들어가니 (0.178)
변경할수없습니다 (0.178)
사용자는 (0.178)
생기던데 (0.178)
생년월일 (0.178)
성인인증하라 (0.178)
성인인증했어요 (0.178)
신규로 (0.178)
인증관련 (0.178)
인증상태 (0.178)
찾아도 (0.178)
탭에서 (0.178)
이제는 (0.178)
쓸만하네요 (0.178)
음성인식이라 (0.178)
시리보단 (0.178)
편리할거 (0.177)
들을수가 (0.177)
감사합니다하고 (0.177)
공짜로 (0.177)
무료인데 (0.177)
바래 (0.177)
잘써야지 (0.177)
1분듣기 (0.177)


호출기능이 (0.102)
검색되면 (0.102)
인터넷이 (0.102)
음량 (0.102)
10분정도만 (0.102)
1시간동안 (0.102)
1시간울리는데 (0.102)
기능좀주세요ㅠㅠ (0.102)
민원들어왔네요ㅜㅜ (0.102)
비웠더니 (0.102)
설정할수있게하든가 (0.102)
알람소리가 (0.102)
외출일을 (0.102)
울리게 (0.102)
울린다고 (0.102)
휴가차 (0.102)
가사 (0.102)
마찬가지네요 (0.102)
숫자 (0.101)
샐리라고 (0.101)
같은거나 (0.101)
제스처 (0.101)
멜론이나 (0.101)
뮤직메이트는 (0.101)
배로 (0.101)
번거로우시겠지만 (0.101)
생깁니다 (0.101)
소리바다도 (0.101)
주세요ㅠㅠ (0.101)
지원에 (0.101)
1234번 (0.101)
어플리케이션과 (0.101)
멜론에서만 (0.101)
뮤직네이트에서 (0.101)
알람노래 (0.101)
있게하지말고 (0.101)
플레이리스트 (0.101)
만들어주신 (0.101)
세개를 (0.101)
요청하여 (0.101)
통합하여 (0.101)
feat (0.101)
김범수님 (0.101)
내려애 (0.101)
넉살 (0.101)
대답해요 (0.101)
때뮨에 (0.101)
범수님이 (0.101)
복식호흡 (0.101)
부드럽게 (0.101)
안따라불러도 (0.101)
왜지 (0.101)
요긴하게 (0.101)
운전하다가 (0.101)
주겠지 (0.101)
텀이 (0.101)
필라멘트 (0.101)
2시간가량 (0.101)
3번정도 (0.101)
고객빠른개선바래요 (0.101)
고객센터확인해보니 (0.101)
말했지만 (0.101)
무료쿠폰등록후 (0.101)
바로주문만되고 (0.101)
불가하단다 (0.101)
성공했으나 (0.101)
시도후 (0.101)
안된다네요 (0.101)
예약주문이 (0.101)
주문량이많아 (0.101)
주문이 (0.101)
주문이밀릴경우 (0.101)
주문접수 (0.101)
주

뮤직으로 (0.049)
함께 (0.049)
보호자 (0.049)
누구미니를 (0.049)
괜찮은것 (0.049)
내가봐서는 (0.049)
매우만족한다 (0.049)
여자친구도 (0.049)
이맵을 (0.049)
좋아해서 (0.049)
2개의 (0.049)
wave와 (0.049)
같은걸 (0.049)
느낀건데 (0.049)
단축버튼 (0.049)
받게하면 (0.049)
상세 (0.049)
설정중에 (0.049)
어떨까요 (0.049)
웨이브나 (0.049)
에디션을 (0.049)
안되는거죠 (0.048)
금융에소 (0.048)
우리은행도 (0.048)
해줘요잉 (0.048)
노럭해주세요 (0.048)
베스트를 (0.048)
문제 (0.048)
비서로 (0.048)
조용해줘 (0.048)
종료를 (0.048)
콘텐츠 (0.048)
했어요 (0.048)
ㅎ우리집은 (0.047)
기가지니여서 (0.047)
듣는중 (0.047)
무료100회 (0.047)
확실한 (0.047)
미니에 (0.047)
기능이랑 (0.047)
기분에따라 (0.047)
들려주는 (0.047)
선곡해서 (0.047)
편해졌어요 (0.047)
lg는 (0.047)
끄기 (0.047)
눌렀을때 (0.047)
되든 (0.047)
불편하더라구요 (0.047)
홈버튼을 (0.047)
만족할것같아요 (0.047)
연동되는것보다 (0.047)
음악앱에서 (0.047)
틀어줘하면 (0.047)
해주신다면 (0.047)
노래불러줘하면 (0.047)
연락을 (0.047)
위해서 (0.047)
괜찮은거 (0.047)
바라는점이 (0.047)
서비스인 (0.047)
서비스치고 (0.047)
실행되도록 (0.047)
편해질것 (0.047)
되는데요 (0.046)
질문하면 (0.046)
같은소리인데 (0.046)
같은자리 (0.046)
못알아듣는거같아요 (0.046)
샀을때보다 (0.046)
없네요ㅠㅜ (0.046)
구매해 (0.046)
듣게되면 (0.046)
듯해 (0.046)
벅스는 (0.046)
벅스와 (0.04

브라운스피커2주째 (-0.000)
아침까지 (-0.000)
안하다니요 (-0.000)
어이가없어서 (-0.000)
어이없어 (-0.000)
지니뮤직메뉴도 (-0.000)
지니뮤직연동해서 (-0.000)
클로바전용스피커가 (-0.000)
판매를 (-0.000)
하더군요 (-0.000)
돈을 (-0.000)
되세요 (-0.000)
리뷰쓰는데 (-0.000)
받다니 (-0.000)
사도 (-0.000)
수정되고 (-0.000)
시키는데 (-0.000)
아니에요 (-0.000)
연결따위 (-0.000)
인테리어 (-0.000)
자동연결 (-0.000)
지금30분째 (-0.000)
할꺼 (-0.000)
돈받아쳐먹는지 (-0.000)
순삭이라 (-0.000)
안된다는둥 (-0.000)
알수 (-0.000)
어플이고뭐고 (-0.000)
오류메세지도 (-0.000)
읽지도 (-0.000)
지경입니다 (-0.000)
나니까 (-0.000)
네트워크지연이라면서 (-0.000)
노래들으려고 (-0.000)
누구어플로 (-0.000)
뭐지요 (-0.000)
실행시켜도 (-0.000)
쓰는건데 (-0.000)
얘기가 (-0.000)
전엔 (-0.000)
주파수 (-0.000)
기사읽을때 (-0.000)
나은게 (-0.000)
낙제점 (-0.000)
뉴스검색만 (-0.000)
답게 (-0.000)
만드시길 (-0.000)
못미침 (-0.000)
비교했을 (-0.000)
스피커하고는 (-0.000)
악을 (-0.000)
야구경기 (-0.000)
음악듣는 (-0.000)
이용권하나로 (-0.000)
입력이 (-0.000)
잘못잡아서 (-0.000)
정녕 (-0.000)
조차 (-0.000)
초기버전이라고는 (-0.000)
초반에 (-0.000)
팔아먹을거 (-0.000)
핀번호 (-0.000)
하나도없음 (-0.000)
한참걸림 (-0.000)
후발주자인데도 (-0.000)
갤노8인데 (-0.000)
동화같은거 (-0.000)
렉이 (-0.000)
무한로딩에 (-0.000)
안하겠는데 (

...

몇달전 (-0.038)
비교해봤는데 (-0.038)
뽐x라는 (-0.038)
사이트에 (-0.038)
시간같은 (-0.038)
시도하는 (-0.038)
알아먹던게 (-0.038)
앞단어는 (-0.038)
올렸었는데 (-0.038)
있네 (-0.038)
천천히 (-0.038)
철썩같이 (-0.038)
캐치하는 (-0.038)
테스트해보니 (-0.038)
활동하던 (-0.038)
회전중인데 (-0.038)
답답해죽겠어요 (-0.039)
잠깐되더니 (-0.039)
나왔는데 (-0.039)
남기면 (-0.039)
높은데 (-0.039)
다른곡이 (-0.039)
마이 (-0.039)
말해봤자 (-0.039)
못알아먹네요 (-0.039)
샐리사용 (-0.039)
선호도도 (-0.039)
스테이션에서 (-0.039)
아티스트도 (-0.039)
애매해서 (-0.039)
연주곡이라 (-0.039)
인내심 (-0.039)
자장가가 (-0.039)
재울때 (-0.039)
제목만 (-0.039)
최상위에 (-0.039)
틀어달래도 (-0.039)
틀어댑니다 (-0.039)
해놨고 (-0.039)
호출해서 (-0.039)
안받고 (-0.039)
아니라고 (-0.039)
bbc (-0.039)
cnn이나 (-0.039)
kbs1 (-0.039)
가까워 (-0.039)
나오는게별로없어요 (-0.039)
누구아이디어인가요 (-0.039)
뉴스채널좀 (-0.039)
대답을해요 (-0.039)
따로제작된뉴스음성은 (-0.039)
떠들어서 (-0.039)
라디오방송을 (-0.039)
목록도 (-0.039)
아리랑라디오는 (-0.039)
열어주셔요 (-0.039)
영어라디오 (-0.039)
청취가능한 (-0.039)
피곤해요 (-0.039)
확인할수있게해주세요 (-0.039)
끄거나 (-0.040)
되고ᆢ왜 (-0.040)
듣기할때도 (-0.040)
반복하다보면 (-0.040)
안내는 (-0.040)
안되구요ᆞ다시 (-0.040)
안들립니다ᆞ폰으로 (-0.040)
이런거죠 (-0.040)
재생상

얼굴을 (-0.064)
우아하게 (-0.064)
음성명령기능을 (-0.064)
음악소리에 (-0.064)
음악켜두면 (-0.064)
직장과 (-0.064)
질러서 (-0.064)
쳐야하는데 (-0.064)
타자를 (-0.064)
파묻고 (-0.064)
판국입니다 (-0.064)
해야할 (-0.064)
화장실 (-0.064)
소리도 (-0.065)
알아듣지도 (-0.065)
앱이라고 (-0.065)
호출할 (-0.065)
구입하라고 (-0.065)
동안은 (-0.065)
뜻이죠 (-0.065)
막상 (-0.065)
멈춘다는 (-0.065)
바꿉니다 (-0.065)
상품을 (-0.065)
순간 (-0.065)
스트리밍 (-0.065)
아리를 (-0.065)
안내하지만 (-0.065)
연동하고 (-0.065)
연동할때 (-0.065)
btv연결 (-0.065)
tv에 (-0.065)
기종때문인건 (-0.065)
다른경로로 (-0.065)
라고만 (-0.065)
사용자설정 (-0.065)
서비스설정 (-0.065)
서비스입니다 (-0.065)
접금해서 (-0.065)
표시된 (-0.065)
개선해서 (-0.065)
답이없고 (-0.065)
돌리는방법을 (-0.065)
못알아들음ㅋㅋ (-0.065)
알려주시던지 (-0.065)
오답500 (-0.065)
전버젼으로 (-0.065)
하루빨리 (-0.065)
로그인만 (-0.065)
안삼 (-0.065)
와이파이만 (-0.065)
잘되던거 (-0.065)
초기화 (-0.065)
필요한게 (-0.065)
네이버뮤직으로 (-0.065)
이미 (-0.066)
비서라고 (-0.066)
못하는게 (-0.066)
현재 (-0.066)
재생이 (-0.066)
기분이 (-0.066)
밖으로 (-0.066)
왔는데 (-0.066)
데이타를 (-0.066)
소진시키는 (-0.066)
안시켰는데 (-0.066)
5까지밖에 (-0.066)
수정해주셈 (-0.066)
업데이트이후 (-0.066)
최대 (-0.066)
셋팅하려고 (-0.066)
중

재생할수있게끔 (-0.108)
저같이 (-0.108)
참고하셔야할듯 (-0.108)
해주시지 (-0.108)
마치 (-0.108)
뒤에 (-0.108)
내놓는게 (-0.108)
대화조차 (-0.108)
반성하셔야 (-0.108)
상품화한거 (-0.108)
스피커랍시고 (-0.108)
하시겠지만 (-0.108)
차이가 (-0.108)
권한 (-0.108)
등이 (-0.108)
초록색 (-0.108)
흰색등만 (-0.108)
재생목록에 (-0.108)
같은곡만 (-0.108)
시다바리인줄 (-0.108)
아놔 (-0.108)
알겠더라 (-0.108)
연관노래재생좀 (-0.108)
재생하냐 (-0.108)
해봐라 (-0.108)
운영을 (-0.108)
날씨도 (-0.108)
날씨나 (-0.108)
키즈캐슬 (-0.108)
되옵니다 (-0.108)
미국인데 (-0.108)
낫다 (-0.109)
못한 (-0.109)
소찬휘 (-0.109)
잘못하네요 (-0.109)
코요태 (-0.109)
있다고 (-0.109)
드럽게 (-0.109)
2만원짜리 (-0.109)
되야되는데 (-0.109)
맞나 (-0.109)
보다못함 (-0.109)
와이파이연결이 (-0.109)
잡히지도 (-0.109)
검색결과만 (-0.109)
검색만도 (-0.109)
구글어시스턴스까지 (-0.109)
띄워주는 (-0.109)
비교가 (-0.109)
열악합니다 (-0.109)
음성인식에서 (-0.109)
분명히 (-0.110)
j5 (-0.110)
미니버전 (-0.110)
버전 (-0.110)
변경안됨 (-0.110)
사고싶은데 (-0.110)
생긴거 (-0.110)
안내위치 (-0.110)
알려주셈 (-0.110)
월이라도 (-0.110)
이뻐보여서 (-0.110)
출시일 (-0.110)
켰음 (-0.110)
다른거 (-0.110)
맞게썼는데 (-0.110)
번호도 (-0.110)
실패하나요 (-0.110)
대답하네요 (-0.110)
로만 (-0.110)
버전도 (-0.110)
사왔는데 (-0.11

반품안돼니 (-0.168)
사용못해도 (-0.168)
유의바람 (-0.168)
로그아웃되더니 (-0.168)
로그인도안되고 (-0.168)
이럽니다 (-0.168)
일주일째 (-0.168)
틀렸다고 (-0.168)
매달 (-0.168)
오늘도 (-0.168)
보호자인증으로 (-0.169)
보호자인증했는데 (-0.169)
그전에도 (-0.169)
내부문제로 (-0.169)
안되었는데 (-0.169)
안된다네 (-0.169)
고객센터 (-0.169)
검색기능도 (-0.169)
기능이라니 (-0.169)
만드시느라 (-0.169)
만들어노셨네요 (-0.169)
쓰잘데기없는 (-0.169)
은행 (-0.169)
음식 (-0.169)
주문같은 (-0.169)
전원을 (-0.170)
업데이트되더니 (-0.170)
성질나서 (-0.170)
날씨물어보라고하고 (-0.170)
맙시다 (-0.170)
생각없이 (-0.170)
스킵하게 (-0.170)
안넘어가짐 (-0.170)
해주던가 (-0.170)
있게해주세요 (-0.170)
누구가 (-0.170)
음악틀면 (-0.170)
여자 (-0.170)
오류나면 (-0.170)
간이 (-0.170)
아는건지 (-0.170)
장난감으로 (-0.170)
큰건지 (-0.170)
하다니 (-0.170)
된거임 (-0.170)
불안정한건 (-0.170)
옆에서 (-0.170)
아는게 (-0.171)
그냥그렇다 (-0.171)
글구 (-0.171)
내가볼때 (-0.171)
따로있다니 (-0.171)
멜론으로음악결제를신청했는데 (-0.171)
시건방지게느껴짐 (-0.171)
실수 (-0.171)
아리아용이 (-0.171)
해줬음 (-0.171)
광고에비해 (-0.171)
기계가안좋은건지 (-0.171)
미흡함 (-0.171)
연동이안됨 (-0.171)
유플iot플러그 (-0.171)
잘못함 (-0.171)
잘연동됨 (-0.171)
업데이트해도 (-0.171)
못한다 (-0.171)
눌러야 (-0.171)
없는페이지라는데요 (-0.171)
열기하면

주고싶다 (-0.274)
귀가 (-0.274)
어플업뎃이후로 (-0.274)
작동에 (-0.274)
네이버랑 (-0.274)
음악검색은 (-0.274)
없다고함 (-0.274)
안드로이드 (-0.275)
기가지니버디기능 (-0.275)
시켜주세요 (-0.275)
샐리에서 (-0.275)
있어도 (-0.275)
라디오에서 (-0.275)
괜찮은데 (-0.275)
중에서 (-0.275)
잡음이 (-0.275)
개별로 (-0.275)
안되는지 (-0.276)
없네요 (-0.276)
그런거죠 (-0.276)
때매 (-0.276)
늦어요 (-0.276)
어플에서는 (-0.276)
a8 (-0.276)
연결기기를찾을수없다고계속뜨네요 (-0.276)
아직 (-0.276)
됐다고하는데 (-0.277)
사용하겠어요 (-0.277)
설치했는데요 (-0.277)
시일 (-0.277)
다음으로 (-0.277)
불를 (-0.277)
안넘어가져요 (-0.277)
정한뒤 (-0.277)
그러지 (-0.277)
어린이라 (-0.277)
올레닷컴 (-0.277)
기능좀요 (-0.277)
하든말든하죠 (-0.277)
5분만에삭제 (-0.277)
알아듣질 (-0.277)
장난 (-0.277)
삭제함 (-0.278)
더똑똑합니다 (-0.278)
멍청해요 (-0.278)
전화도 (-0.278)
종료 (-0.278)
되어 (-0.278)
알람소리 (-0.278)
카카오야 (-0.278)
아직도 (-0.278)
갖다 (-0.279)
빡치네요 (-0.279)
정도로 (-0.279)
여기 (-0.279)
않됨 (-0.279)
생각하고 (-0.279)
개선해주셨으면 (-0.279)
목록 (-0.279)
버튼 (-0.280)
해결이 (-0.281)
머지 (-0.281)
절차에서 (-0.281)
해야된다는 (-0.281)
하나요 (-0.281)
기가지니앱에 (-0.282)
노래찾는기능이 (-0.282)
노래찾을 (-0.282)
없던데 (-0.282)
지니뮤직에선 (-0.282)
차트만 (-0.282)
켜졌다 

In [128]:
with open(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\sentiment_dictionary\senti_dictionary.txt', 'w', encoding='utf-8') as f:
    for word, coef in sorted_coefficients:
        f.write(vocablist[word]+'\t'+str(coef)+'\n')
f.close()

In [44]:
import re 
sent_dict = {}
with open(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\sentiment_dictionary\senti_dictionary.txt', 'r', encoding='utf-8') as f:
    for i in f.readlines():
        word, score = i.split('\t')
        score = re.sub('\n', '', score)
        sent_dict[word] = float(score)
f.close()

In [45]:
sent_dict

{'좋아요': 2.9527606222300258,
 '좋은데': 2.251907199235719,
 'good': 1.8491984037755202,
 '좋네요': 1.8169588665472602,
 '좋음': 1.7246830948415321,
 '좋습니다': 1.6709440517110479,
 '최고': 1.6201384608282334,
 '감사합니다': 1.5747820031620254,
 '너무좋아요': 1.4445872355522489,
 '근데': 1.326636953086338,
 '있습니다': 1.3044598557124025,
 '아주좋아요': 1.2346810317447146,
 '좋아': 1.182774871944742,
 '추가해주세요': 1.179038258497266,
 '다좋은데': 1.1551426728890044,
 '편해요': 1.0784052977699103,
 '좋고': 1.07381743810381,
 '감사': 1.064778009755287,
 '같아요': 1.0454006563436928,
 '좋은': 1.0383077433608938,
 '굿굿': 1.002114292889251,
 '제가': 0.9887222692231149,
 '최고의': 0.9868726272030411,
 '좋다': 0.9793795124540977,
 '좋네': 0.9563002345487744,
 '잘쓰고': 0.9314680738978124,
 '최고에요': 0.913119329239129,
 '쵝오': 0.9059027051729514,
 '좋겠네요': 0.9053388771286865,
 '아리아': 0.9017962345978487,
 '그런데': 0.8899175643339942,
 'nugu': 0.8885840914486435,
 '조아요': 0.8870554536820461,
 '빅스비보다': 0.8825332568884904,
 '좋겠어요': 0.8727164780987425,
 '있었으면': 0.86555737862

In [179]:
sent_dict = {}
for word, coef in sorted_coefficients:
    sent_dict[vocablist[word]] = coef
sent_dict

{'좋아요': 2.9526365397046006,
 '좋은데': 2.2516683380580385,
 'good': 1.8489078844881726,
 '좋네요': 1.8166668418466116,
 '좋음': 1.7243950817841833,
 '좋습니다': 1.6710564911966266,
 '최고': 1.6198631536759365,
 '감사합니다': 1.5744289983787203,
 '너무좋아요': 1.4445207357646836,
 '근데': 1.3259092550497764,
 '있습니다': 1.3068808073905174,
 '아주좋아요': 1.2344609762629262,
 '좋아': 1.1826321553619878,
 '추가해주세요': 1.1788512496661605,
 '다좋은데': 1.1552033651041658,
 '편해요': 1.078104430783355,
 '좋고': 1.0731711592158804,
 '감사': 1.0642715017158606,
 '같아요': 1.045057518697363,
 '좋은': 1.0376158331857264,
 '굿굿': 1.0017470375943998,
 '제가': 0.9891542349881555,
 '최고의': 0.9869545265957445,
 '좋다': 0.9791651319408108,
 '좋네': 0.9564719608267904,
 '잘쓰고': 0.9311742073965216,
 '최고에요': 0.9128656680345015,
 '쵝오': 0.9054240777772419,
 '좋겠네요': 0.9050567615154744,
 '아리아': 0.9013321454268016,
 '그런데': 0.8910966324881633,
 'nugu': 0.8882264637177401,
 '조아요': 0.8870544501824127,
 '빅스비보다': 0.8821219668651791,
 '좋겠어요': 0.8725736657432833,
 '있었으면': 0.8653

# <br>
# wordcloud

## 감성사전으로 긍정, 부정, 중립 문헌 나누기

In [8]:
df = pd.read_csv(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\naver_blog\naverclova_speaker_blog.csv', encoding='UTF-16')
texts = list(df['text'].dropna())
texts = list(map(lambda x: x.replace('\n', ' ').replace('\r', ' '), texts))

### RogisticRegression 이용하여 나누기

### 문장 별로 나누었을 때

In [51]:
import nltk.tokenize
sentences = list(map(nltk.tokenize.sent_tokenize, texts))

[['침대 옆 공간은 좁지만 놓아야 할 것들이 꽤 많아서 늘 비좁은데, 침대맡에 하나 두고 저녁 생활을 도울 괜찮은 기능의 블루투스 스피커 + 라디오 알람시계를 소개합니다.',
  '블루투스 스피커 / LED 알람시계 / 라디오 / 충전용USB 아이러브(iLuv) 모닝콜 2, 이 제품은 책상이나 주방, 특히 침대맡에 두기 딱 좋은 다기능 라디오 알람시계입니다.',
  '딱히 라디오라고 정의하기는 애매한데 생긴 것이 라디오 같으니까...ㅎㅎ 크기는 사진에서 보이는 것보다 좀 작습니다.',
  '한 뼘 조금 안되는 아담 사이즈.',
  '침대 옆 탁자나 싱크대, 혹은 책상 앞에 올려 놓기 좋은 크기예요.',
  '밤에 조명을 끈 방에서는 LED시계가 필요하죠.',
  '그런데 시계의 화면이 엄청 큰!!',
  '!아주 잘 보여요.',
  '^^[ LED 밝기조절 ]작은 불빛에도 수면에 방해 받는 분들이 있더라고요.',
  '제 아내처럼.',
  '그런 분들은 LED 밝기를 5단계로 조절하시면 됩니다.',
  '(스누즈 버튼) ▼ \xa0● Radio : 정해진 시간에 라디오가 켜집니다\xa0● Buzzer : 설정시간에 뚜뚜 부저음이 울려요아침에 두 번의 알람을 설정해 울릴 수 있고 두 가지 종류 중 선택합니다.',
  '아침에는 두 개를 시간차 설정해서 확실히 깨면 되겠죠?',
  '라디오로 깰 분위기 잡고 부저 알람으로 발딱 일어나기.',
  '(혹은 거꾸로...) ㅎㅎ 요게 라디오 알람시계를 찾는 이유 아니겠어요?※ 스누즈 버튼을 누르면 약 9분 후 알람이 다시 울립니다 주파수를 탐색해 듣거나 6개 채널 주파수가 저장이 됩니다.',
  'Sleep버튼으로 일정 시간 후 자동으로 꺼지도록 설정할 수 있어요.',
  '(15, 30, 45, 60, 90분) ※ 안테나는 긴 "선"이 뒤에 달려 있습니다.',
  '펼쳐서 벽에 붙이거나 적절히 위치를 조절하면 신호가 잘 잡혀요.',
  '스마트폰과 블루투스 연결로 MP3나 애플뮤직, 벅스 팟캐스트 같은 서비스의 스트

In [67]:
lr_pos, lr_neg =  [],[]
for sentence in sentences:
    lr_pos_sen, lr_neg_sen = [], []
    test_tf_features = tf_vectorizer.transform(sentence)
#    print(test_tf_features)
    pred_labels = lr.predict(test_tf_features)
    
    for num, score in enumerate(pred_labels):
        if score == 1:
            lr_pos_sen.append(sentence[num])
        else:
            lr_neg_sen.append(sentence[num])
    lr_pos.append(lr_pos_sen)
    lr_neg.append(lr_neg_sen)

In [75]:
print(len(lr_pos)), print(len(lr_neg))

3405
3405


(None, None)

In [70]:
pd.DataFrame(sum(lr_pos,[]))

,0
0,"침대 옆 공간은 좁지만 놓아야 할 것들이 꽤 많아서 늘 비좁은데, 침대맡에 하나 두..."
1,블루투스 스피커 / LED 알람시계 / 라디오 / 충전용USB 아이러브(iLuv) ...
2,딱히 라디오라고 정의하기는 애매한데 생긴 것이 라디오 같으니까...ㅎㅎ 크기는 사진...
3,한 뼘 조금 안되는 아담 사이즈.
4,"침대 옆 탁자나 싱크대, 혹은 책상 앞에 올려 놓기 좋은 크기예요."
5,밤에 조명을 끈 방에서는 LED시계가 필요하죠.
6,그런데 시계의 화면이 엄청 큰!!
7,!아주 잘 보여요.
8,^^[ LED 밝기조절 ]작은 불빛에도 수면에 방해 받는 분들이 있더라고요.
9,제 아내처럼.


In [74]:
pd.DataFrame(sum(lr_neg,[]))
# for t in pd.DataFrame(sum(lr_neg,[]))[0]:
#     print(t)

,0
0,ㅋㅋ<체크사항> 반드시 전원을 연결해야 작동을 하는 제품입니다.
1,"어학부터 책, 트렌드, 정보 등 음성 위주의 들을꺼리가 가득!"
2,♬ 교양이 솔솔~( 다운로드 : 애플 앱스토어 / 구글 플레이 ) ■ Mixer B...
3,MP3 준비 없이도 쓱~♬ (앱 설명 포스팅 보기)Mixerbox 3 ( 애플 앱스...
4,WAVE 를 입수한 김에 큰 맘먹고 리뷰를 한 번 써보려다가 결국 근사한 리뷰작성 ...
5,간신히 와이파이 설정 단계를 넘어 간 이후 AI 스피커 활용사례로 제일 많이 꼽히는...
6,"""MY > 이용권 구매 > 내 이용권 정보"" 페이지의 맨 하단에 있다....ㅠㅠ정말..."
7,구글 홈구글의 인공지능 서비스를 바탕으로 다양한 서비스를 진행 중입니다.자세한 내용...
8,애플 홈팟올해 12월에 판매가 될 예상으로 이미 다양한 정보들이 다 공개가 되었습니...
9,이미 상용화되고 출시도 되어 있고한국어에 100% 호환성이 있기 때문에 사용할 때는...


### 문헌 별로 나누었을 때

In [58]:
lr_pos, lr_neg =  [],[]
for num, score in enumerate(pred_labels):
    if score == 1:
        lr_pos.append(texts[num])
    else:
        lr_neg.append(texts[num])

In [20]:
len(lr_pos)

2032

In [22]:
lr_pos[:5]

['침대 옆 공간은 좁지만 놓아야 할 것들이 꽤 많아서 늘 비좁은데, 침대맡에 하나 두고 저녁 생활을 도울 괜찮은 기능의 블루투스 스피커 + 라디오 알람시계를 소개합니다.  블루투스 스피커 / LED 알람시계 / 라디오 / 충전용USB 아이러브(iLuv) 모닝콜 2, 이 제품은 책상이나 주방, 특히 침대맡에 두기 딱 좋은 다기능 라디오 알람시계입니다. 딱히 라디오라고 정의하기는 애매한데 생긴 것이 라디오 같으니까...ㅎㅎ 크기는 사진에서 보이는 것보다 좀 작습니다.  한 뼘 조금 안되는 아담 사이즈. 침대 옆 탁자나 싱크대, 혹은 책상 앞에 올려 놓기 좋은 크기예요.  밤에 조명을 끈 방에서는 LED시계가 필요하죠. 그런데 시계의 화면이 엄청 큰!!!아주 잘 보여요. ^^[ LED 밝기조절 ]작은 불빛에도 수면에 방해 받는 분들이 있더라고요. 제 아내처럼. 그런 분들은 LED 밝기를 5단계로 조절하시면 됩니다. (스누즈 버튼) ▼ \xa0● Radio : 정해진 시간에 라디오가 켜집니다\xa0● Buzzer : 설정시간에 뚜뚜 부저음이 울려요아침에 두 번의 알람을 설정해 울릴 수 있고 두 가지 종류 중 선택합니다.  아침에는 두 개를 시간차 설정해서 확실히 깨면 되겠죠? 라디오로 깰 분위기 잡고 부저 알람으로 발딱 일어나기. (혹은 거꾸로...) ㅎㅎ 요게 라디오 알람시계를 찾는 이유 아니겠어요?※ 스누즈 버튼을 누르면 약 9분 후 알람이 다시 울립니다 주파수를 탐색해 듣거나 6개 채널 주파수가 저장이 됩니다. Sleep버튼으로 일정 시간 후 자동으로 꺼지도록 설정할 수 있어요. (15, 30, 45, 60, 90분) ※ 안테나는 긴 "선"이 뒤에 달려 있습니다. 펼쳐서 벽에 붙이거나 적절히 위치를 조절하면 신호가 잘 잡혀요.  스마트폰과 블루투스 연결로 MP3나 애플뮤직, 벅스 팟캐스트 같은 서비스의 스트리밍 음악을 들을 수 있어요. 해외 영어방송 채널이나 외국어 회화 들으시는 분들도 있으시죠.  [블루투스 페어링] 밤에 스마트폰 충전은 필수인데 USB충전단

In [21]:
len(lr_neg)

1373

In [23]:
lr_neg[:5]

['WAVE 를 입수한 김에 큰 맘먹고 리뷰를 한 번 써보려다가 결국 근사한 리뷰작성 근처에도 가지 못하고 사용준비 하는 과정에서 사소한 몇 가지 걸림돌 때문에 QA 항목 몇 가지 발견하고 중도 포기.... wave는 기가속도(olleh_GiGA_WiFi_4B16)는 몇 번을 시도해도 설정되지 않아서 결국 일반속도 와이파이(olleh_WiFi_4B16)으로 무선랜 설정을 마칠 수 있었다. 간신히 와이파이 설정 단계를 넘어 간 이후 AI 스피커 활용사례로 제일 많이 꼽히는 음악 감상을 위해 박스에 동봉된 네이버뮤직 선물쿠폰을 입력하다가 또 벽에 걸리고 말았다.이 단계에서는 갑자기 WAVE가 아닌 Naver Music 앱 QA로 전환된다~쿠폰에 적혀 있는 간단한 설명 " 모바일: 네이버뮤직앱 또는 웹 > 메뉴 > 이용권 > 쿠폰등록" 아이폰 네이버뮤직 앱에서는 "이용권 구매 / 내 이용권" 메뉴 어디에서도 쿠폰 등록 버튼은 찾을 수 없다... 아이폰 네이버뮤직 앱에서 쿠폰 등록 메뉴 찾기에 실패하고 보유 중인 안드로이드폰 네이버뮤직 앱에 들어가서 찾아보니....쿠폰에 적혀 있는 " 모바일: 네이버뮤직앱 또는 웹 > 메뉴 > 이용권 > 쿠폰등록" 와는 조금 다른 단계로 찾긴 찾을 수 있었다."MY > 이용권 구매 > 내 이용권 정보" 페이지의 맨 하단에 있다....ㅠㅠ정말 블로그에 글을 적고 있는 이 순간까지도 설마 아이폰 앱이라고 해서 쿠폰등록 메뉴가 없을 리가..싶어서 다시 한 번 확인해봤다. 어딘가에 있는데 내가 황당한 착각을 하고 있는 것은 아닐 지?\xa0하지만 열번을 양보해서 어딘가에 쿠폰등록메뉴가 있긴 있더라도 적어도 쿠폰 안내문은 좀 더 정확하게 표현해줘야 하는 것 아닐까 싶다...',
 '최근 등장한 새로운 용어바로 스마트 스피커입니다.간단히 설명하면 음성인식이 가능해서 인터넷의 인공지능 서버를 통해 사용자의 요구에 대해 적절하게 반응해주는 장치입니다.하지만 가장 먼저는 음악 재생들의 기본적인 기능과 함께 함께 연결되어 있는 홈 네크워크를 적절하게 

### sentiment dictionary 이용하여 나누기

In [46]:
tf_vectorizer, tf_features = tf_extractor(texts)
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]

In [47]:
# 각 feature 들의 감성 점수 list (감성사전에 없는 feature = 0)
sent_score = []
for i in vocablist:
    if i in sent_dict:
        sent_score.append(sent_dict[i])
    else:
        sent_score.append(0)

In [48]:
# 각 문헌의 점수를 계산
tf_score = []
for i in tf_features:
    tf_score.append(i.dot(sent_score)[0])
tf_score

[16.503492987043913,
 -5.997952963612364,
 0.5634197401186769,
 -4.0075183326530865,
 21.883267509417756,
 -0.7450948700129161,
 3.370555175226662,
 -29.432248760299643,
 5.106500916442445,
 12.085252988401644,
 -12.757437910475323,
 -0.6328930921061193,
 2.490329224779809,
 -0.9271787146144134,
 -15.104426972324632,
 7.373917864793675,
 -11.69013738178129,
 -5.330093455071581,
 4.996017020404211,
 -6.226069167343789,
 3.5322651388350064,
 7.138603539104491,
 2.8386760214406537,
 -5.928354130525635,
 -16.696745734028614,
 -4.17613264858293,
 -1.7600481357720967,
 8.358776156142511,
 -4.914627581857023,
 -2.7850863004763236,
 -32.363544268906445,
 3.0966924964507285,
 -11.534721979692113,
 22.72634797982358,
 -10.092595918498818,
 3.253946689379105,
 -3.460767764880818,
 -4.29908554726237,
 -5.848964831508074,
 3.3457090624423214,
 -3.943167797542783,
 -3.4646856458959547,
 -1.717739368404886,
 4.517915351777255,
 -0.5638340801130165,
 23.2635015022667,
 -2.06691098532715,
 12.678208926

In [49]:
pos = []
neg = []
neu = []
for num, score in enumerate(tf_score):
    if score > 0:
        pos.append(texts[num])
    elif score < 0:
        neg.append(texts[num])
    else:
        neu.append(texts[num])

### Dead kernel... 일단 저장..

In [17]:
with open('pos_texts.txt', 'w', encoding='utf-8') as p:
    for i in pos:
        p.write(i+'\n')
p.close()
len(pos)

1848

In [49]:
with open('neg_texts.txt', 'w', encoding='utf-8') as n:
    for i in neg:
        n.write(i+'\n')
n.close()
len(neg)

1555

In [50]:
with open('neu_texts.txt', 'w', encoding='utf-8') as ne:
    for i in neu:
        ne.write(i+'\n')
ne.close()
len(neu)

2

In [38]:
pos = []
with open('pos_texts.txt', 'r', encoding='utf-8') as p:
    for i in p.readlines():
        pos.append(i.split('\n')[0])
len(pos)

1848

In [39]:
pos[:5]

['침대 옆 공간은 좁지만 놓아야 할 것들이 꽤 많아서 늘 비좁은데, 침대맡에 하나 두고 저녁 생활을 도울 괜찮은 기능의 블루투스 스피커 + 라디오 알람시계를 소개합니다.  블루투스 스피커 / LED 알람시계 / 라디오 / 충전용USB 아이러브(iLuv) 모닝콜 2, 이 제품은 책상이나 주방, 특히 침대맡에 두기 딱 좋은 다기능 라디오 알람시계입니다. 딱히 라디오라고 정의하기는 애매한데 생긴 것이 라디오 같으니까...ㅎㅎ 크기는 사진에서 보이는 것보다 좀 작습니다.  한 뼘 조금 안되는 아담 사이즈. 침대 옆 탁자나 싱크대, 혹은 책상 앞에 올려 놓기 좋은 크기예요.  밤에 조명을 끈 방에서는 LED시계가 필요하죠. 그런데 시계의 화면이 엄청 큰!!!아주 잘 보여요. ^^[ LED 밝기조절 ]작은 불빛에도 수면에 방해 받는 분들이 있더라고요. 제 아내처럼. 그런 분들은 LED 밝기를 5단계로 조절하시면 됩니다. (스누즈 버튼) ▼ \xa0● Radio : 정해진 시간에 라디오가 켜집니다\xa0● Buzzer : 설정시간에 뚜뚜 부저음이 울려요아침에 두 번의 알람을 설정해 울릴 수 있고 두 가지 종류 중 선택합니다.  아침에는 두 개를 시간차 설정해서 확실히 깨면 되겠죠? 라디오로 깰 분위기 잡고 부저 알람으로 발딱 일어나기. (혹은 거꾸로...) ㅎㅎ 요게 라디오 알람시계를 찾는 이유 아니겠어요?※ 스누즈 버튼을 누르면 약 9분 후 알람이 다시 울립니다 주파수를 탐색해 듣거나 6개 채널 주파수가 저장이 됩니다. Sleep버튼으로 일정 시간 후 자동으로 꺼지도록 설정할 수 있어요. (15, 30, 45, 60, 90분) ※ 안테나는 긴 "선"이 뒤에 달려 있습니다. 펼쳐서 벽에 붙이거나 적절히 위치를 조절하면 신호가 잘 잡혀요.  스마트폰과 블루투스 연결로 MP3나 애플뮤직, 벅스 팟캐스트 같은 서비스의 스트리밍 음악을 들을 수 있어요. 해외 영어방송 채널이나 외국어 회화 들으시는 분들도 있으시죠.  [블루투스 페어링] 밤에 스마트폰 충전은 필수인데 USB충전단

In [40]:
neg = []
with open('neg_texts.txt', 'r', encoding='utf-8') as n:
    for i in n.readlines():
        neg.append(i.split('\n')[0])
len(neg)

1555

In [41]:
neg[:5]

['WAVE 를 입수한 김에 큰 맘먹고 리뷰를 한 번 써보려다가 결국 근사한 리뷰작성 근처에도 가지 못하고 사용준비 하는 과정에서 사소한 몇 가지 걸림돌 때문에 QA 항목 몇 가지 발견하고 중도 포기.... wave는 기가속도(olleh_GiGA_WiFi_4B16)는 몇 번을 시도해도 설정되지 않아서 결국 일반속도 와이파이(olleh_WiFi_4B16)으로 무선랜 설정을 마칠 수 있었다. 간신히 와이파이 설정 단계를 넘어 간 이후 AI 스피커 활용사례로 제일 많이 꼽히는 음악 감상을 위해 박스에 동봉된 네이버뮤직 선물쿠폰을 입력하다가 또 벽에 걸리고 말았다.이 단계에서는 갑자기 WAVE가 아닌 Naver Music 앱 QA로 전환된다~쿠폰에 적혀 있는 간단한 설명 " 모바일: 네이버뮤직앱 또는 웹 > 메뉴 > 이용권 > 쿠폰등록" 아이폰 네이버뮤직 앱에서는 "이용권 구매 / 내 이용권" 메뉴 어디에서도 쿠폰 등록 버튼은 찾을 수 없다... 아이폰 네이버뮤직 앱에서 쿠폰 등록 메뉴 찾기에 실패하고 보유 중인 안드로이드폰 네이버뮤직 앱에 들어가서 찾아보니....쿠폰에 적혀 있는 " 모바일: 네이버뮤직앱 또는 웹 > 메뉴 > 이용권 > 쿠폰등록" 와는 조금 다른 단계로 찾긴 찾을 수 있었다."MY > 이용권 구매 > 내 이용권 정보" 페이지의 맨 하단에 있다....ㅠㅠ정말 블로그에 글을 적고 있는 이 순간까지도 설마 아이폰 앱이라고 해서 쿠폰등록 메뉴가 없을 리가..싶어서 다시 한 번 확인해봤다. 어딘가에 있는데 내가 황당한 착각을 하고 있는 것은 아닐 지?\xa0하지만 열번을 양보해서 어딘가에 쿠폰등록메뉴가 있긴 있더라도 적어도 쿠폰 안내문은 좀 더 정확하게 표현해줘야 하는 것 아닐까 싶다...',
 '최근 등장한 새로운 용어바로 스마트 스피커입니다.간단히 설명하면 음성인식이 가능해서 인터넷의 인공지능 서버를 통해 사용자의 요구에 대해 적절하게 반응해주는 장치입니다.하지만 가장 먼저는 음악 재생들의 기본적인 기능과 함께 함께 연결되어 있는 홈 네크워크를 적절하게 

In [4]:
neu = []
with open('neu_texts.txt', 'r', encoding='utf-8') as ne:
    for i in ne.readlines():
        neu.append(i.split('\n')[0])
len(neu)

2

### positive document

In [42]:
from tqdm import tqdm_notebook

In [43]:
# kornoun extractor로 명사 추출 for 미등록 단어
from kornounextractor.noun_extractor import extract
extract_texts = list(map(extract, pos))
extract_texts = sum(extract_texts, [])

In [6]:
with open("./dic_pos.txt", 'w', encoding='UTF-8') as f:
    for i in extract_texts:
        f.write(i+'\tNNG\n')

In [ ]:
# 미등록단어 지정하여 Komoran 형태소 분석 
import konlpy.tag
komoran = konlpy.tag.Komoran()

dicpath = './dic_pos.txt'
komoran = konlpy.tag.Komoran(userdic= dicpath)
komo_nouns = list(map(komoran.nouns, pos))

In [ ]:
def remove_one_length(x):
    return len(x)>1

filtered_nouns = []
for noun in komo_nouns:
    filtered_noun = list(filter(remove_one_length, noun))
    filtered_nouns.append(filtered_noun)

In [ ]:
# 불용어 제거
with open(r'C:\Users\sormd\Desktop\수업_1학기\기계학습\과제\stopwordsKor.txt', 'r', encoding='utf8') as f:
    stopwords = f.read()

def remove_stopwords(x):
    for word in x:
        if word in stopwords:
            x.remove(word)
    return(x)

cleaned_nouns = list(map(remove_stopwords, filtered_nouns))

In [54]:
import collections
sum_nouns = sum(cleaned_nouns,[])
count_nouns = collections.Counter(sum_nouns)

In [55]:
collections.OrderedDict(count_nouns.most_common())

OrderedDict([('클로바', 5347),
             ('스피커', 4878),
             ('네이버', 3780),
             ('사용', 3093),
             ('인공지능', 2760),
             ('가능', 2667),
             ('기능', 2386),
             ('서비스', 1870),
             ('AI', 1684),
             ('프렌즈', 1586),
             ('음악', 1567),
             ('제품', 1453),
             ('연결', 1445),
             ('보다', 1384),
             ('함께', 1367),
             ('IoT', 1178),
             ('부터', 1160),
             ('정보', 1115),
             ('설정', 1108),
             ('이용', 1089),
             ('확인', 1026),
             ('활용', 983),
             ('기술', 978),
             ('LG', 956),
             ('필요', 934),
             ('스마트', 930),
             ('편리', 925),
             ('음성', 916),
             ('오늘', 915),
             ('노래', 913),
             ('영어', 908),
             ('아이들', 848),
             ('브라운', 844),
             ('검색', 838),
             ('블루투스', 832),
             ('버튼', 830),
             ('콘텐츠', 823),
   

In [56]:
sorted(count_nouns.items(), key=lambda i: i[1])

[('충전용USB', 1),
 ('LED시계', 1),
 ('Radio', 1),
 ('Buzzer', 1),
 ('설정시간', 1),
 ('부저음', 1),
 ('시간차', 1),
 ('부저', 1),
 ('Sleep버튼', 1),
 ('영어방송', 1),
 ('충전상태', 1),
 ('연애통화', 1),
 ('꿀기능', 1),
 ('AAA배터리', 1),
 ('설정사항', 1),
 ('용도예요', 1),
 ('검색유입', 1),
 ('와글와글', 1),
 ('음악취향', 1),
 ('스푼', 1),
 ('spoon', 1),
 ('라디오판', 1),
 ('클립네이버', 1),
 ('음성판', 1),
 ('팟빵팟캐스트', 1),
 ('Box3', 1),
 ('Vox음악', 1),
 ('사운드클라우드', 1),
 ('Mixerbox', 1),
 ('evermusic', 1),
 ('pro클라우드', 1),
 ('mp3몽땅', 1),
 ('cnspower', 1),
 ('은행업무', 1),
 ('입출금', 1),
 ('외부약속', 1),
 ('출타', 1),
 ('텔레비시청', 1),
 ('온라인고스톱', 1),
 ('베타버젼', 1),
 ('인식자체', 1),
 ('구글음성', 1),
 ('배우이름', 1),
 ('검색하기위', 1),
 ('사드린건데', 1),
 ('거주환경', 1),
 ('백 투 더 퓨처', 1),
 ('로버트', 1),
 ('저메키스', 1),
 ('바이센테니얼맨', 1),
 ('콜럼버스', 1),
 ('면면', 1),
 ('컬처', 1),
 ('CUMI', 1),
 ('포켓린트', 1),
 ('국제가전박람회', 1),
 ('1924년', 1),
 ('산업전시', 1),
 ('가전전시회', 1),
 ('smartphone4europe', 1),
 ('scoop', 1),
 ('자연언어', 1),
 ('Thinkstock', 1),
 ('shutterstock', 1),
 ('음향학', 1),
 ('지멘스', 1),
 ('보쉬', 1),
 

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 그래프에 retina display 적용
#%config InlineBackend.figure_format = 'retina'

# 나눔고딕 설치
!apt -qq -y install fonts-nanum > /dev/null
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)

In [ ]:
wordcloud = WordCloud(
    font_path = fontpath,
    width = 800,
    height = 600,
    #stopwords = lines,
    background_color="white"
)

wordcloud = wordcloud.generate_from_frequencies(word_freq)

plt.figure(figsize=(15, 11))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()